In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [5]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price-whole'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star-small a-star-small-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [6]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=pc&crid=34HHY4CDIXH43&sprefix=%2Caps%2C691&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

C:\Users\ARYAN KHRISTI\AppData\Local\Temp\ipykernel_60524\3342232890.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [7]:
amazon_df

,title,price,rating,reviews,availability
0,"CyberPowerPC Gamer Master Gaming PC, AMD Ryzen...",$719.99,4.6 out of 5 stars,"1,402 ratings",In Stock
1,"CyberPowerPC Gamer Xtreme VR Gaming PC, Intel ...","$1,358.94",4.5 out of 5 stars,49 ratings,In Stock
2,LXZ Desktop PC Computer Ryzen 5-5600GT 8 core ...,$719.99,4.6 out of 5 stars,6 ratings,Not Available
3,"CyberPowerPC Gamer Xtreme VR Gaming PC, Intel ...",$989.99,4.3 out of 5 stars,59 ratings,In Stock
4,iBUYPOWER Y40 PRO Black Gaming PC Desktop Comp...,"$2,069.99",3.8 out of 5 stars,6 ratings,In Stock
5,"Gaming PC Desktop Computer, Ryzen 7 5700X,RTX ...",$829.99,4.3 out of 5 stars,38 ratings,Not Available
6,"Gaming PC Desktop Computer,Intel Core i5 13400...","$1,799.99",5.0 out of 5 stars,3 ratings,Not Available
7,LXZ Desktop PC Computer AMD Ryzen 7-5700G 8 co...,$87.97,4.4 out of 5 stars,11 ratings,Not Available
8,Reatan A6 Mini Gaming PC AMD Oculink Ryzen 7 7...,"$1,974.99",4.5 out of 5 stars,104 ratings,Not Available
9,GMKtec Mini PC AMD Ryzen 7 5825U (Turbo 4.5GHz...,,4.6 out of 5 stars,127 ratings,Not Available
